In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

## Setup

In [3]:
from math import isclose
from ipyexperiments import IPyExperimentsCPU
import ipyexperiments
import re
from utils.text import check_report_strings, check_data, check_report_cpu, consume_cpu_ram_128mb, consume_gpu_ram_256mb, locals_unset, check_reclaimed, check_defined, check_undefined, print_output

In [4]:
"""test_version"""
assert ipyexperiments.__version__, "version check"

'test_version'

## Basic container test


In [5]:
%%capture output
locals_unset(['exp1', 'x1', 'x2'])

exp1 = IPyExperimentsCPU(cl_enable=False)

x2 = consume_cpu_ram_128mb()
x1 = consume_cpu_ram_128mb()
check_defined(['x1', 'x2'])

#exp1.finish()
del exp1
check_undefined(['x1', 'x2'])

# cleanup
locals_unset(['exp1', 'x1', 'x2'])

In [6]:
output = str(output)
print_output(output)
check_reclaimed(output)

Captured output:
| 
| *** Experiment started with the CPU-only backend
| 
| 
| *** Current state:
| RAM:     Used     Free    Total        Util
| CPU:    1,056   87,856  128,696 MB   0.82% 
| 
| 
| 
| IPyExperimentsCPU: Finishing
| 
| *** Experiment finished in 00:00:00 (elapsed wallclock time)
| 
| *** Newly defined local variables:
| Deleted: x1, x2
| 
| *** Experiment memory:
| RAM: Consumed       Reclaimed
| CPU:      256      255 MB ( 99.99%)
| 
| *** Current state:
| RAM:     Used     Free    Total        Util
| CPU:    1,056   87,849  128,696 MB   0.82% 
| 
| 
| 



## Test stats data and selective vars preservation

In [7]:
%%capture output
exp2 = IPyExperimentsCPU(cl_enable=False) 
x1 = consume_cpu_ram_128mb()

cpu_data = exp2.data.cpu
assert cpu_data
print(cpu_data)

exp2.keep_var_names('cpu_data')

x2 = consume_cpu_ram_128mb() 

cpu_data = exp2.data.cpu
print(cpu_data)

check_defined(['x1', 'x2', 'cpu_data'])


data = exp2.finish() # finish experiment
cpu_data_final = data.cpu
print("\nNumerical data:\n", cpu_data_final)

check_defined(['cpu_data_final'])
check_undefined(['x1', 'x2'])

In [8]:
output = str(output)
print_output(output)
check_reclaimed(output)
check_data(output, cpu_data_final)

Captured output:
| 
| *** Experiment started with the CPU-only backend
| 
| 
| *** Current state:
| RAM:     Used     Free    Total        Util
| CPU:    1,056   87,850  128,696 MB   0.82% 
| 
| 
| IPyExperimentMemory(consumed=134017024, reclaimed=0, available=91983151104)
| IPyExperimentMemory(consumed=268312576, reclaimed=0, available=91848683520)
| 
| IPyExperimentsCPU: Finishing
| 
| *** Experiment finished in 00:00:00 (elapsed wallclock time)
| 
| *** Newly defined local variables:
| Deleted: x1, x2
| Kept:    cpu_data
| 
| *** Experiment memory:
| RAM: Consumed       Reclaimed
| CPU:      255      255 MB ( 99.98%)
| 
| *** Current state:
| RAM:     Used     Free    Total        Util
| CPU:    1,056   87,848  128,696 MB   0.82% 
| 
| 
| 
| Numerical data:
|  IPyExperimentMemory(consumed=268312576, reclaimed=268251136, available=92116037632)
| 



## Using the context manager

If you want to put all cells into one, you could simplify the experiment even further by using its context manager.

In [9]:
%%capture output
with IPyExperimentsCPU(cl_enable=False):
    x1 = consume_cpu_ram_128mb()
    x2 = consume_cpu_ram_128mb()
    
check_undefined(['x1', 'x2'])

In [10]:
output = str(output)
print_output(output)
check_reclaimed(output)

Captured output:
| 
| *** Experiment started with the CPU-only backend
| 
| 
| *** Current state:
| RAM:     Used     Free    Total        Util
| CPU:    1,056   87,849  128,696 MB   0.82% 
| 
| 
| 
| IPyExperimentsCPU: Finishing
| 
| *** Experiment finished in 00:00:00 (elapsed wallclock time)
| 
| *** Newly defined local variables:
| Deleted: x1, x2
| 
| *** Experiment memory:
| RAM: Consumed       Reclaimed
| CPU:      255      255 MB (100.00%)
| 
| *** Current state:
| RAM:     Used     Free    Total        Util
| CPU:    1,056   87,848  128,696 MB   0.82% 
| 
| 
| 



In [11]:
%%capture output
with IPyExperimentsCPU(cl_enable=False) as exp: 
    x1 = consume_cpu_ram_128mb()
    z = "some data"
    x2 = consume_cpu_ram_128mb()
    exp.keep_var_names('z')
print(z)
check_defined(['z'])
check_undefined(['x1', 'x2'])

In [12]:
output = str(output)
print_output(output)
check_reclaimed(output)

Captured output:
| 
| *** Experiment started with the CPU-only backend
| 
| 
| *** Current state:
| RAM:     Used     Free    Total        Util
| CPU:    1,056   87,848  128,696 MB   0.82% 
| 
| 
| 
| IPyExperimentsCPU: Finishing
| 
| *** Experiment finished in 00:00:00 (elapsed wallclock time)
| 
| *** Newly defined local variables:
| Deleted: x1, x2
| Kept:    z
| 
| *** Experiment memory:
| RAM: Consumed       Reclaimed
| CPU:      255      255 MB (100.00%)
| 
| *** Current state:
| RAM:     Used     Free    Total        Util
| CPU:    1,056   87,848  128,696 MB   0.82% 
| 
| 
| some data
| 



## Deleting previously defined variables

Here we test that we **cannot** correctly detect and delete variables used in the scope of the experiment, but which were already defined prior to the experiment.

In [13]:
%%capture output
x3 = 10
x4 = 20
with IPyExperimentsCPU(cl_enable=False): 
    x1 = consume_cpu_ram_128mb()
    x2 = consume_cpu_ram_128mb()
    x3 = 10
    x4 = 10000
# x3 is the same value, so we can't tell the difference whether it was created before the experiment or not
# x4 is different from previous value, but it could have been modified by some function indirectly and not used directly in the experiment
check_defined(['x3', 'x4'])
# x3 is newly defined, so it gets deleted
check_undefined(['x1', 'x2'])

In [14]:
output = str(output)
print_output(output)
check_reclaimed(output)

Captured output:
| 
| *** Experiment started with the CPU-only backend
| 
| 
| *** Current state:
| RAM:     Used     Free    Total        Util
| CPU:    1,056   87,848  128,696 MB   0.82% 
| 
| 
| 
| IPyExperimentsCPU: Finishing
| 
| *** Experiment finished in 00:00:00 (elapsed wallclock time)
| 
| *** Newly defined local variables:
| Deleted: x1, x2
| 
| *** Experiment memory:
| RAM: Consumed       Reclaimed
| CPU:      255      255 MB (100.00%)
| 
| *** Current state:
| RAM:     Used     Free    Total        Util
| CPU:    1,056   87,848  128,696 MB   0.82% 
| 
| 
| 



Same but with the CellLogger enabled

In [15]:
%%capture output
x3 = 10
x4 = 20
with IPyExperimentsCPU(cl_enable=True): 
    x1 = consume_cpu_ram_128mb()
    x2 = consume_cpu_ram_128mb()
    x3 = 10
    x4 = 10000
# x3 is the same value, so we can't tell the difference whether it was created before the experiment or not
# x4 is different from previous value, but it could have been modified by some function indirectly and not used directly in the experiment
check_defined(['x3', 'x4'])
# x3 is newly defined, so it gets deleted
check_undefined(['x1', 'x2'])

In [16]:
output = str(output)
print_output(output)
check_reclaimed(output)

Captured output:
| 
| *** Experiment started with the CPU-only backend
| 
| 
| *** Current state:
| RAM:     Used     Free    Total        Util
| CPU:    1,056   87,848  128,696 MB   0.82% 
| 
| 
| ･ RAM:  △Consumed    △Peaked    Used Total | Exec time 0:00:00.028
| ･ CPU:        255          0      1,311 MB |
| 
| IPyExperimentsCPU: Finishing
| 
| *** Experiment finished in 00:00:00 (elapsed wallclock time)
| 
| *** Newly defined local variables:
| Deleted: x1, x2
| 
| *** Experiment memory:
| RAM: Consumed       Reclaimed
| CPU:      255      255 MB (100.00%)
| 
| *** Current state:
| RAM:     Used     Free    Total        Util
| CPU:    1,056   87,813  128,696 MB   0.82% 
| 
| 
| 



In [ ]:
%%javascript # prevent committing an unsaved notebook
IPython.notebook.save_notebook()